In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name = 'Udacity_project'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131457
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-131457


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'ML-project1'

try :
    cpu_cluster = ComputeTarget(workspace = ws,
                               name = cpu_cluster_name)
    print("Using previously created cpu_cluster")
except ComputeTargetException :
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',
                                                          max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output = True)    

Using previously created cpu_cluster

Running


In [28]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil

# Specify parameter sampler # OWN CODE
ps = RandomParameterSampling({
    "--C": uniform(0.0, 1),
    "--max_iter":choice(50, 100, 150, 200, 250)
})

# Specify a Policy ### OWN code 
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#script_folder = './training'
#os.makedirs(script_folder, exist_ok=True)

#shutil.copy('./train.py', script_folder)

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(source_directory = script_folder, 
              compute_target = cpu_cluster, 
              vm_size = 'STANDARD_D2_V2', 
              vm_priority = 'lowpriority',
              entry_script = 'train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling = ps,
                                     policy = policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_concurrent_runs = 3,
                                     max_total_runs = 50)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ad925e55-56ad-45b6-8266-aba412800b62
Web View: https://ml.azure.com/experiments/Udacity_project/runs/HD_ad925e55-56ad-45b6-8266-aba412800b62?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-131457/workspaces/quick-starts-ws-131457

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-23T10:45:34.074197][API][INFO]Experiment created<END>\n"<START>[2020-12-23T10:45:35.1640736Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-23T10:45:38.560556][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2020-12-23T10:45:38.874530][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run.get_details()['runDefinition']['arguments'])
print('Id of the best run: ', best_run.id)
print('Accuracy of the best run:', best_run_metrics['Accuracy'])

# save best model
best_run.register_model(model_name = 'best_hyperdrive_model', model_path = 'outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.TabularDataset.from_delimited_files(url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
# x_train, x_test, y_train, y_test = SKLearn.train_test_split(x, y, test_size = 0,2, random_state=30)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=x,
    label_column_name=y, # most likely incorrect, need to check the column
    n_cross_validations=5)


In [2]:
# Submit your automl run

### YOUR CODE HERE ###
experiment_auto = Experiment(ws, 'automl_remote')
remote_run = experiment_auto.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_autorun = remote_run.get_best_run_by_primary_metric()
best_autorun_metrics = best_autorun.get_metrics()
print(best_autorun.get_details()['runDefinition']['arguments'])
print('Id of the best run: ', best_autorun.id)
print('Accuracy of the best run:', best_autorun_metrics['Accuracy'])

model_auto = remote_run.register_model(model_name = 'best_automl_model', model_path = 'outputs/model_auto.joblib') #first option
# best_run.download_file("/outputs/model2.joblib", "logreg_auto.joblib") #second option